In [ ]:
!pip install evaluate bert_score

In [2]:
import os
import pandas as pd
from PIL import Image
from tqdm import tqdm
import torch
from transformers import BlipProcessor, BlipForQuestionAnswering, BartTokenizer, BartForConditionalGeneration
from PIL import Image
import torch
import pandas as pd
from tqdm import tqdm
from sklearn.model_selection import train_test_split
# import evaluate
# from bert_score import score as bert_score

from transformers import (
    BlipForQuestionAnswering,
    BlipProcessor
)

model = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base")
processor = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")

In [3]:
df = pd.read_csv('/kaggle/input/combined-vqa/flattened_vqa_with_metadata.csv')
df = df.sample(frac=1, random_state=42).copy()

In [ ]:
# import os
# import io
# import torch
# import warnings
# from PIL import Image
# import contextlib
# import pandas as pd


# # Assuming processor and model are already initialized and on CUDA
# # image_base_path should be the root of your image folder
# image_base_path = "/kaggle/input/abo-small/small"

# skipped = 0
# skip_logs = []

# for idx, row in df.iterrows():
#     question = row["question"]
#     gt_answer = str(row["answer"]).strip().lower()
#     image_rel_path = row["image_path"]

#     full_path = image_rel_path  # Already contains full relative path
#     if not os.path.exists(full_path):
#         skip_logs.append(f"[SKIP] Image not found: {full_path}")
#         skipped += 1
#         continue

#     try:
#         image = Image.open(full_path).convert("RGB")
#     except Exception as e:
#         skip_logs.append(f"[SKIP] Failed to load image: {full_path} | Error: {e}")
#         skipped += 1
#         continue

#     with warnings.catch_warnings():
#         warnings.simplefilter("ignore")
#         with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
#             inputs = processor(images=image, text=question, return_tensors="pt")
#             output = model.generate(**inputs, max_new_tokens=10)
#             pred_answer = processor.decode(output[0], skip_special_tokens=True).strip().lower()

#     print(f"Q: {question}\nPredicted: {pred_answer} | Ground Truth: {gt_answer}\n")


In [ ]:
!pip install -U bitsandbytes

In [4]:
!pip install peft

In [5]:
# import torch
# from peft import get_peft_model, LoraConfig, TaskType




# # LoRA configuration
# lora_config = LoraConfig(
#     r=4,
#     lora_alpha=16,
#     target_modules=["q", "v"],  # 'q' and 'v' are common; confirm exact target modules if needed
#     lora_dropout=0.1,
#     bias="none",
#     task_type=TaskType.SEQ_2_SEQ_LM
# )

# # Apply LoRA
# model = get_peft_model(model, lora_config)

# # Print number of trainable parameters
# def print_trainable_params(model):
#     trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
#     total = sum(p.numel() for p in model.parameters())
#     print(f"Trainable params: {trainable:,} / {total:,} ({100 * trainable / total:.2f}%)")

# print_trainable_params(model)

from peft import get_peft_model, LoraConfig, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=16,
    lora_alpha=16,
    target_modules=["projection", "value", "key", "query", "fc1", "fc2", "qkv"],
    lora_dropout=0.1,
    bias="none",
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 5,898,240 || all params: 390,570,812 || trainable%: 1.5102


In [6]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
import requests
from transformers import BlipProcessor, BlipForQuestionAnswering
import torch
from PIL import Image
from torch.utils.data import DataLoader
from tqdm import tqdm
import pickle


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Dataset class modification for CSV format
class CSVVQADataset(torch.utils.data.Dataset):
    def __init__(self, dataframe, processor):
        self.data = dataframe
        self.processor = processor

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        item = self.data.iloc[idx]
        image = Image.open(item['image_path']).convert("RGB")
        text = item['question']
        answer = item['answer']
        if isinstance(answer, list):
            answer = " ".join(map(str, answer))  # convert list items to str if needed
        elif not isinstance(answer, str):
            answer = str(answer)
        
        if isinstance(text, list):
            text = " ".join(map(str, text))  # convert list items to str if needed
        elif not isinstance(text, str):
            text = str(text)
            
        encoding = self.processor(
            image, 
            text, 
            max_length = 64,
            padding="max_length", 
            truncation=True, 
            return_tensors="pt"
        )
        
        labels = self.processor.tokenizer(
            answer, 
            max_length= 64, 
            padding="max_length", 
            return_tensors='pt'
        )
        
        encoding["labels"] = labels["input_ids"].squeeze()
        for k,v in encoding.items():  
            encoding[k] = v.squeeze()
        return encoding

# Load and split data
train_df, valid_df = train_test_split(df, test_size=0.2, random_state=42)

# Create datasets
train_dataset = CSVVQADataset(train_df, processor)
valid_dataset = CSVVQADataset(valid_df, processor)

# Create dataloaders
batch_size = 3
train_dataloader = DataLoader(
    train_dataset, 
    batch_size=batch_size, 
    shuffle=True,  # Enable shuffling
    pin_memory=True
)
valid_dataloader = DataLoader(
    valid_dataset, 
    batch_size=batch_size, 
    shuffle=False, 
    pin_memory=True
)


optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
scaler = torch.cuda.amp.GradScaler()

# Training Loop
min_eval_loss = float("inf")
tracking_information = []
patience = 3
early_stopping_hook = 0

for epoch in range(2):
    model.train()
    train_loss = 0
    for batch in tqdm(train_dataloader, desc=f"Epoch {epoch+1} Training"):
        for k in batch:
            batch[k] = batch[k].to(device)
        with torch.cuda.amp.autocast():
            outputs = model(**batch)
            loss = outputs.loss
        optimizer.zero_grad()
        scaler.scale(loss).backward()
        scaler.step(optimizer)
        scaler.update()
        train_loss += loss.item()
    
    model.eval()
    eval_loss = 0
    with torch.no_grad():
        for batch in tqdm(valid_dataloader, desc=f"Epoch {epoch+1} Validation"):
            for k in batch:
                batch[k] = batch[k].to(device)
            with torch.cuda.amp.autocast():
                outputs = model(**batch)
                loss = outputs.loss
            eval_loss += loss.item()
    
    train_loss /= len(train_dataloader)
    eval_loss /= len(valid_dataloader)
    tracking_information.append((train_loss, eval_loss))
    print(f"Epoch {epoch+1}: Train Loss = {train_loss:.4f}, Eval Loss = {eval_loss:.4f}")

    if eval_loss < min_eval_loss:
        min_eval_loss = eval_loss
        model.save_pretrained("Model/blip-lora-saved")
        model.save_pretrained("/kaggle/working/blip-lora-saved")
        print("Model saved.")
        early_stopping_hook = 0
    else:
        early_stopping_hook += 1
        if early_stopping_hook >= patience:
            print("⏹Early stopping.")
            break

pickle.dump(tracking_information, open("tracking_info.pkl", "wb"))
print("🎉 Finetuning complete!")


/tmp/ipykernel_101/3676337499.py:85: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
Epoch 1 Training:   0%|          | 0/42447 [00:00<?, ?it/s]/tmp/ipykernel_101/3676337499.py:99: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
We strongly recommend passing in an `attention_mask` since your input_ids may be padded. See https://huggingface.co/docs/transformers/troubleshooting#incorrect-output-when-padding-tokens-arent-masked.
Epoch 1 Validation:   0%|          | 0/10612 [00:00<?, ?it/s]/tmp/ipykernel_101/3676337499.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
Epoch 1 Validation: 100%|██████████| 10612/10612 [28:54<00:00,  6.12it/s]


Epoch 1: Train Loss = 8.4115, Eval Loss = 8.3859
Model saved.


Epoch 2 Validation: 100%|██████████| 10612/10612 [28:57<00:00,  6.11it/s]


Epoch 2: Train Loss = 8.3376, Eval Loss = 8.3277
Model saved.
🎉 Finetuning complete!


In [ ]:
import os
import torch
import pandas as pd
from PIL import Image
from tqdm import tqdm
# import evaluate
from sklearn.model_selection import train_test_split
from transformers import ViltProcessor, ViltForQuestionAnswering
# from transformers import BartTokenizer, BartForConditionalGeneration
import csv
from pathlib import Path

# Prepare output CSV
output_path = Path("vqa_test_predictions_vilt.csv")
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load ViLT processor and model
processor = ViltProcessor.from_pretrained("dandelin/vilt-b32-finetuned-vqa")
model = ViltForQuestionAnswering.from_pretrained("dandelin/vilt-b32-finetuned-vqa").to(device)

# Load dataset
df = pd.read_csv("/kaggle/input/d/sasisnigdhay/combined-vqa/flattened_vqa_with_metadata.csv")
df_sampled = df.sample(frac=0.3, random_state=42).copy()

# Train-test split (80% train, 20% test)
train_df, test_df = train_test_split(df_sampled, test_size=0.2, random_state=42)
test_df = test_df.reset_index(drop=True)

# # Load metrics
# bertscore = evaluate.load("bertscore")
# bleu = evaluate.load("bleu")
# rouge = evaluate.load("rouge")

# # Load BART model for BARTScore
# bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-cnn")
# bart_model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn").to(device)

# def compute_bartscore(preds, refs):
#     scores = []
#     for pred, ref in zip(preds, refs):
#         input_ids = bart_tokenizer.encode(pred, return_tensors="pt").to(device)
#         with torch.no_grad():
#             outputs = bart_model.generate(input_ids, max_length=20, num_beams=4)
#             decoded = bart_tokenizer.decode(outputs[0], skip_special_tokens=True)
#             scores.append(1.0 if decoded.lower() == ref.lower() else 0.0)
#     return scores

# Evaluation on test set
predictions, references = [], []
correct = 0

for idx, row in tqdm(test_df.iterrows(), total=len(test_df), desc="Evaluating ViLT on test set"):
    try:
        image = Image.open(row['image_path']).convert("RGB")
        question = row['question']
        gt_answer = row['answer'].lower()

        inputs = processor(image, question, return_tensors="pt").to(device)
        with torch.no_grad():
            outputs = model(**inputs)
            logits = outputs.logits
            pred_id = logits.argmax(-1).item()
            pred_answer = model.config.id2label[pred_id].lower()

        predictions.append(pred_answer)
        references.append(gt_answer)

        if pred_answer == gt_answer:
            correct += 1
    except Exception as e:
        print(f"Failed on {row['image_path']}: {e}")
        predictions.append("")
        references.append("")
    with open(output_path, mode="a", newline="", encoding="utf-8") as f:
        writer = csv.writer(f)
        writer.writerow([row['image_path'], question, gt_answer, pred_answer])
    print("predictions saved to csv")

In [ ]:
import os
import io
import torch
import warnings
from PIL import Image
import contextlib
import pandas as pd


# Limit to first 5 rows
df = df.head(5)

# Assuming processor and model are already initialized and on CUDA
# image_base_path should be the root of your image folder
image_base_path = "/kaggle/input/abo-small/small"

skipped = 0
skip_logs = []

for idx, row in df.iterrows():
    question = row["question"]
    gt_answer = str(row["answer"]).strip().lower()
    image_rel_path = row["image_path"]

    full_path = image_rel_path  # Already contains full relative path
    if not os.path.exists(full_path):
        skip_logs.append(f"[SKIP] Image not found: {full_path}")
        skipped += 1
        continue

    try:
        image = Image.open(full_path).convert("RGB")
    except Exception as e:
        skip_logs.append(f"[SKIP] Failed to load image: {full_path} | Error: {e}")
        skipped += 1
        continue

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
            inputs = processor(images=image, text=question, return_tensors="pt")
            output = model.generate(**inputs, max_new_tokens=10)
            pred_answer = processor.decode(output[0], skip_special_tokens=True).strip().lower()

    print(f"Q: {question}\nPredicted: {pred_answer} | Ground Truth: {gt_answer}\n")


In [ ]:
!pip install transformers datasets peft accelerate bitsandbytes

In [ ]:
import os
import pandas as pd
from datasets import Dataset, Image

# Load the CSV file
df = pd.read_csv("/kaggle/input/d/sasisnigdhay/combined-vqa/flattened_vqa_with_metadata.csv")
df = df.sample(frac=0.001, random_state=42).copy()

# Rename columns to match expected format
df = df.rename(columns={"image_path": "image", "question": "question", "answer": "answer"})

# Convert to Hugging Face Dataset
dataset = Dataset.from_pandas(df)

# Cast the 'image' column to Image type
dataset = dataset.cast_column("image", Image())

# Split into training and validation sets
split_dataset = dataset.train_test_split(test_size=0.2)
train_dataset = split_dataset["train"]
eval_dataset = split_dataset["test"]

In [ ]:
import torch
from transformers import ViltProcessor, ViltForQuestionAnswering
from peft import get_peft_model, LoraConfig, TaskType

# Load processor and model
model_name = "dandelin/vilt-b32-finetuned-vqa"
processor = ViltProcessor.from_pretrained(model_name)
model = ViltForQuestionAnswering.from_pretrained(model_name)

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query", "value", "key"],
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_CLS
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)


In [ ]:
from torch.utils.data import DataLoader

def collate_fn(batch):
    images = [item["image"] for item in batch]
    questions = [item["question"] for item in batch]
    answers = [item["answer"] for item in batch]

    # Process inputs
    encoding = processor(images=images, text=questions, return_tensors="pt", padding=True, truncation=True)

    # Map answers to label IDs
    label2id = model.config.label2id
    labels = [label2id.get(ans, label2id["unknown"]) for ans in answers]
    encoding["labels"] = torch.tensor(labels)

    return encoding


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./vilt-lora-vqa",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    learning_rate=5e-5,
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
    save_total_limit=2,
    remove_unused_columns=False,
    fp16=torch.cuda.is_available(),
    label_names=["labels"],
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    data_collator=collate_fn,
)


In [ ]:
trainer.train()

In [ ]:
model.save_pretrained("vilt-lora-vqa")
processor.save_pretrained("vilt-lora-vqa")